In [1]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in c:\users\sedulous\anaconda3\lib\site-packages (0.22.1)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np

In [2]:
import sklearn
sklearn.__version__

'0.22.1'

In [3]:
import imblearn

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline  

In [83]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

In [8]:
service=pd.read_csv("E:\\Excel R\\Project Servicenow\\train.csv\\train.csv")

In [9]:
impact=service.impact.copy()

In [10]:
service.drop(['impact'],axis=1,inplace=True)

In [11]:
service.replace('?', np.nan,inplace=True)

In [12]:
service.isna().sum()

Unnamed: 0                0
ID                        0
ID_status                 0
active                    0
count_reassign            0
count_opening             0
count_updated             0
ID_caller                22
opened_by              3328
opened_time               0
Created_by            37174
created_at            37174
updated_by                0
updated_at                0
type_contact              0
location                 54
category_ID              56
user_symptom          23139
Support_group          9994
support_incharge      19302
Doc_knowledge             0
confirmation_check        0
notify                    0
problem_ID            97620
change_request        98512
dtype: int64

In [13]:
cat_new=service.select_dtypes(include=object)

In [14]:
cat_new.drop(["opened_time","updated_at","created_at","problem_ID", "change_request","problem_ID", "change_request"],axis=1,inplace=True)

C:\Users\sedulous\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [15]:
cat_new.drop(["Created_by"],axis=1,inplace=True)

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()

In [17]:
df_temp = cat_new.astype("str").apply(LabelEncoder().fit_transform)


In [18]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99198 entries, 0 to 99197
Data columns (total 12 columns):
ID                  99198 non-null int32
ID_status           99198 non-null int32
ID_caller           99198 non-null int32
opened_by           99198 non-null int32
updated_by          99198 non-null int32
type_contact        99198 non-null int32
location            99198 non-null int32
category_ID         99198 non-null int32
user_symptom        99198 non-null int32
Support_group       99198 non-null int32
support_incharge    99198 non-null int32
notify              99198 non-null int32
dtypes: int32(12)
memory usage: 4.5 MB


In [19]:
df_final = df_temp.where(~cat_new.isna(), cat_new)

In [20]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99198 entries, 0 to 99197
Data columns (total 12 columns):
ID                  99198 non-null int64
ID_status           99198 non-null int64
ID_caller           99176 non-null object
opened_by           95870 non-null object
updated_by          99198 non-null int64
type_contact        99198 non-null int64
location            99144 non-null object
category_ID         99142 non-null object
user_symptom        76059 non-null object
Support_group       89204 non-null object
support_incharge    79896 non-null object
notify              99198 non-null int64
dtypes: int64(5), object(7)
memory usage: 9.1+ MB


In [21]:
new = service.filter(['count_reassign','count_updated'], axis=1)

In [23]:
df=pd.concat([new,df_final],axis=1)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99198 entries, 0 to 99197
Data columns (total 14 columns):
count_reassign      99198 non-null int64
count_updated       99198 non-null int64
ID                  99198 non-null int64
ID_status           99198 non-null int64
ID_caller           99176 non-null object
opened_by           95870 non-null object
updated_by          99198 non-null int64
type_contact        99198 non-null int64
location            99144 non-null object
category_ID         99142 non-null object
user_symptom        76059 non-null object
Support_group       89204 non-null object
support_incharge    79896 non-null object
notify              99198 non-null int64
dtypes: int64(7), object(7)
memory usage: 10.6+ MB


In [25]:
df=pd.concat([df,impact],axis=1)

In [26]:
df['category_ID']= df['category_ID'].fillna(df['category_ID'].value_counts().index[0])

In [27]:
df['ID_caller'] = df['ID_caller'].fillna(df['ID_caller'].value_counts().index[0])

In [28]:
df['location'] = df['location'].fillna(df['location'].value_counts().index[0])

In [29]:
df.isna().sum()

count_reassign          0
count_updated           0
ID                      0
ID_status               0
ID_caller               0
opened_by            3328
updated_by              0
type_contact            0
location                0
category_ID             0
user_symptom        23139
Support_group        9994
support_incharge    19302
notify                  0
impact                  0
dtype: int64

In [30]:
nonull=df[pd.isnull(df['opened_by'])==False]


In [31]:
null=df[pd.isnull(df['opened_by'])]

In [32]:
from sklearn.neighbors import KNeighborsRegressor


In [33]:
rf= RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [34]:
independent_var=['ID', 'ID_status', 'ID_caller', 'updated_by','type_contact', 'location', 'category_ID', 'notify']

In [35]:
rf.fit(nonull[independent_var],nonull['opened_by'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [36]:
prediction=rf.predict(X=null[independent_var])


In [37]:
prediction

array([ 63.32,  38.68,  39.83, ...,  60.72,  90.48, 118.77])

In [38]:
null['opened_by']=prediction.astype(int)


C:\Users\sedulous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
data=nonull.append(null)

In [40]:
data.isna().sum()

count_reassign          0
count_updated           0
ID                      0
ID_status               0
ID_caller               0
opened_by               0
updated_by              0
type_contact            0
location                0
category_ID             0
user_symptom        23139
Support_group        9994
support_incharge    19302
notify                  0
impact                  0
dtype: int64

In [41]:
nonull_1=data[pd.isnull(data['Support_group'])==False]


In [42]:
null_1=data[pd.isnull(data['Support_group'])]

In [43]:
independent_var=['ID', 'ID_status', 'ID_caller', 'updated_by','type_contact', 'location', 'category_ID', 'notify','opened_by']

In [44]:
rf.fit(nonull_1[independent_var],nonull_1['Support_group'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [45]:
prediction_1=rf.predict(X=null_1[independent_var])

In [46]:
null_1['Support_group']=prediction_1.astype(int)

C:\Users\sedulous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
nonull_1=data[pd.isnull(data['Support_group'])==False]

In [48]:
null_1=data[pd.isnull(data['Support_group'])]

In [49]:
independent_var=['ID', 'ID_status', 'ID_caller', 'updated_by','type_contact', 'location', 'category_ID', 'notify','opened_by']

In [50]:
rf.fit(nonull_1[independent_var],nonull_1['Support_group'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [51]:
prediction_1=rf.predict(X=null_1[independent_var])

In [52]:
null_1['Support_group']=prediction_1.astype(int)

C:\Users\sedulous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
data_1=nonull_1.append(null_1)

In [54]:
nonull_2=data_1[pd.isnull(data_1['user_symptom'])==False]

In [55]:
null_2=data_1[pd.isnull(data_1['user_symptom'])]

In [56]:
independent_var=['ID', 'ID_status', 'ID_caller', 'updated_by','type_contact', 'location', 'category_ID', 'notify','opened_by','Support_group']

In [57]:
rf.fit(nonull_2[independent_var],nonull_2['user_symptom'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [58]:
independent_var=['ID', 'ID_status', 'ID_caller', 'updated_by','type_contact', 'location', 'category_ID', 'notify','opened_by','Support_group']

In [59]:
rf.fit(nonull_2[independent_var],nonull_2['user_symptom'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [60]:
prediction_2=rf.predict(X=null_2[independent_var])

In [61]:
null_2['user_symptom']=prediction_2.astype(int)

C:\Users\sedulous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [62]:
data_2=nonull_2.append(null_2)

In [63]:
data_2.isna().sum()

count_reassign          0
count_updated           0
ID                      0
ID_status               0
ID_caller               0
opened_by               0
updated_by              0
type_contact            0
location                0
category_ID             0
user_symptom            0
Support_group           0
support_incharge    19302
notify                  0
impact                  0
dtype: int64

In [64]:
independent_var=['ID', 'ID_status', 'ID_caller', 'updated_by','type_contact', 'location', 'category_ID', 'notify','opened_by','Support_group','user_symptom']

In [65]:
nonull_3=data_2[pd.isnull(data_2['support_incharge'])==False]

In [66]:

null_3=data_2[pd.isnull(data_2['support_incharge'])]

In [67]:
rf.fit(nonull_3[independent_var],nonull_3['support_incharge'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [68]:
prediction_3=rf.predict(X=null_3[independent_var])

In [69]:
null_3['support_incharge']=prediction_3.astype(int)

C:\Users\sedulous\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [70]:
Imputated_data=nonull_3.append(null_3)

In [71]:
Imputated_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99198 entries, 3 to 58285
Data columns (total 15 columns):
count_reassign      99198 non-null int64
count_updated       99198 non-null int64
ID                  99198 non-null int64
ID_status           99198 non-null int64
ID_caller           99198 non-null int64
opened_by           99198 non-null object
updated_by          99198 non-null int64
type_contact        99198 non-null int64
location            99198 non-null int64
category_ID         99198 non-null int64
user_symptom        99198 non-null object
Support_group       99198 non-null object
support_incharge    99198 non-null object
notify              99198 non-null int64
impact              99198 non-null object
dtypes: int64(10), object(5)
memory usage: 12.1+ MB


In [72]:
Imputated_data.drop(['ID_status','type_contact','type_contact'],axis=1,inplace=True)

Ensemble SMOTE TOMEK

In [79]:
clf_uo_1 = DecisionTreeClassifier()

In [80]:
clf_uo_2 = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False)

In [81]:
clf_uo_3= KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='hamming',metric_params=None, n_jobs=None, n_neighbors=9,weights='uniform')

In [84]:
eclf_uo = VotingClassifier(estimators=[('dt', clf_uo_1), ('rf', clf_uo_2), ('knn', clf_uo_3)], voting='soft')

In [85]:
from sklearn.model_selection import cross_val_score

In [86]:
scores = cross_val_score(eclf_uo, Imputated_data.iloc[:,0:11],Imputated_data['impact'], cv=10)

In [87]:
scores

array([0.93891129, 0.95625   , 0.93608871, 0.95393145, 0.94324597,
       0.95080645, 0.95060484, 0.94768145, 0.96471418, 0.96632725])

In [89]:
eclf_uo.fit(Imputated_data.iloc[:,0:11],Imputated_data['impact'])


VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features=None,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
                                                     random_state=None,
   

In [94]:
import pickle


In [96]:
with open('service_model_en.pkl', 'wb') as file:
    pickle.dump(eclf_uo, file)